In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

from cleanlab.multiannotator import get_majority_vote_label, get_label_quality_multiannotator, get_active_learning_scores
from cleanlab.internal.label_quality_utils import get_normalized_entropy

from utils.model_training import fit_predict_proba
from utils.active_learning import setup_next_iter_data, setup_next_iter_data_single, add_new_annotator, add_new_annotator_single, get_idx_to_label

## Load Files

The datafiles can either be generate yourself by running [0_create_data.ipynb](0_create_data.ipynb), or by downloading our pre-generated files using the commands:

```
wget -nc 'https://cleanlab-public.s3.amazonaws.com/ActiveLearning/Benchmark/SingleVsMultiannotator/data.tar.gz'
tar -xf data.tar.gz data/
```

In [ ]:
num_iter = 5
num_rounds = 15
batch_size_to_label = 100

noise_rate_arr = [0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
def get_data(noise_rate):
    multiannotator_labels = pd.DataFrame(
        np.load(f"data/{noise_rate}/multiannotator_labels_labeled.npy")
    )
    single_labels = np.load(f"data/{noise_rate}/single_labels_labeled.npy")

    true_labels_labeled = np.load(f"data/{noise_rate}/true_labels_labeled.npy")
    true_labels_unlabeled = np.load(f"data/{noise_rate}/true_labels_unlabeled.npy")
    true_labels_test = np.load(f"data/{noise_rate}/true_labels_test.npy")

    extra_labels_labeled = np.load(f"data/{noise_rate}/extra_labels_labeled.npy")
    extra_labels_unlabeled = np.load(f"data/{noise_rate}/extra_labels_unlabeled.npy")

    X_labeled = np.load(f"data/{noise_rate}/X_labeled.npy")
    X_unlabeled = np.load(f"data/{noise_rate}/X_unlabeled.npy")
    X_test = np.load(f"data/{noise_rate}/X_test.npy")

    return (
        multiannotator_labels,
        single_labels,
        X_labeled,
        X_unlabeled,
        X_test,
        true_labels_labeled,
        true_labels_unlabeled,
        true_labels_test,
        extra_labels_labeled,
        extra_labels_unlabeled,

    )

In [ ]:
for noise_rate in noise_rate_arr:
    for iter in range(num_iter):
        dirname = f"results/{noise_rate}"
        !mkdir -p $dirname

        model_accuracy_arr = np.full(num_rounds, np.nan)

        (multiannotator_labels, single_labels, X_labeled, X_unlabeled, X_test,
        y_labeled, y_unlabeled, y_test, extra_labels_labeled, 
        extra_labels_unlabeled) = get_data(noise_rate)

        for i in range(num_rounds):
            # get consensus labels
            if i == 0:
                consensus_labels = get_majority_vote_label(multiannotator_labels)
            else:
                # we can use the pred_probs from last round as the best model estimate
                results = get_label_quality_multiannotator(
                    multiannotator_labels, 
                    pred_probs_labeled,
                    calibrate_probs=True,
                )
                consensus_labels = results["label_quality"]["consensus_label"]

            # train model to get out-of-sample predicted probabilites 
            pred_probs, pred_probs_unlabeled = fit_predict_proba(
                ExtraTreesClassifier(),
                X_labeled,
                consensus_labels,
                cv_n_folds=5,
                X_unlabeled=X_unlabeled,
            )

            # train a model on the full set of labeled data to evaluate model accuracy for the current round,
            # this is an optional step for demonstration purposes, in practical applications 
            # you may not have ground truth labels
            model = ExtraTreesClassifier()
            model.fit(X_labeled, consensus_labels)
            pred_labels = model.predict(X_test)
            model_accuracy_arr[i] = np.mean(pred_labels == y_test)

            # compute active learning scores
            active_learning_scores, active_learning_scores_unlabeled = get_active_learning_scores(
                multiannotator_labels, pred_probs, pred_probs_unlabeled
            )

            # get the indices of examples to collect more labels for
            relabel_idx, relabel_idx_unlabeled = get_idx_to_label(
                active_learning_scores=active_learning_scores,
                active_learning_scores_unlabeled=active_learning_scores_unlabeled,
                batch_size_to_label=batch_size_to_label,
            )

            # format the data for the next round of active learning, ie. moving some unlabeled 
            # examples to the labeled pool because we are collecting labels for them
            (
                multiannotator_labels, relabel_idx_combined, X_labeled, X_unlabeled, pred_probs_labeled, 
                pred_probs_unlabeled, extra_labels_labeled, extra_labels_unlabeled,
            ) = setup_next_iter_data(
                multiannotator_labels, relabel_idx, relabel_idx_unlabeled, X_labeled, X_unlabeled, pred_probs, 
                pred_probs_unlabeled, extra_labels_labeled, extra_labels_unlabeled,
            )

            # add a new annotator that provides new labels for the examples with the lowest
            # active learning scores (indices obtained above)
            multiannotator_labels = add_new_annotator(
                multiannotator_labels, extra_labels_labeled, relabel_idx_combined
            )

        np.save(f"results/{noise_rate}/multiannotator_{iter}.npy", model_accuracy_arr)

In [ ]:
for noise_rate in noise_rate_arr:
    for iter in range(num_iter):
        dirname = f"results/{noise_rate}"
        !mkdir -p $dirname

        single_model_accuracy_arr = np.full(num_rounds, np.nan)

        (multiannotator_labels, single_labels, X_labeled, X_unlabeled, X_test,
        y_labeled, y_unlabeled, y_test, _, extra_labels_single) = get_data(noise_rate)

        for i in range(num_rounds):
            # train model to get out-of-sample predicted probabilites 
            pred_probs, pred_probs_unlabeled = fit_predict_proba(
                ExtraTreesClassifier(),
                X_labeled,
                single_labels,
                cv_n_folds=5,
                X_unlabeled=X_unlabeled,
            )

            # train a model on the full set of labeled data to evaluate model accuracy for the current round,
            # this is an optional step for demonstration purposes, in practical applications 
            # you may not have ground truth labels
            # train model on single label
            single_model = ExtraTreesClassifier()
            single_model.fit(X_labeled, single_labels)
            single_pred_labels = single_model.predict(X_test)
            single_model_accuracy_arr[i] = np.mean(single_pred_labels == y_test)

            # compute active learning scores (entropy)
            quality_of_consensus = - get_normalized_entropy(pred_probs_unlabeled)

            # get the indices of examples to collect more labels for
            relabel_idx = np.array([])
            relabel_idx_unlabeled = np.argsort(quality_of_consensus)[:batch_size_to_label]

            # add a new annotator that provides new labels for the examples with the lowest
            # active learning scores (indices obtained above)
            new_single_annotator_labels = add_new_annotator_single(extra_labels_single, relabel_idx_unlabeled)
            single_labels = np.concatenate((single_labels, new_single_annotator_labels))

            # format the data for the next round of active learning, ie. moving some unlabeled 
            # examples to the labeled pool because we are collecting labels for them
            (
                relabel_idx_combined, X_labeled, X_unlabeled, pred_probs_labeled, 
                pred_probs_unlabeled, extra_labels_single
            ) = setup_next_iter_data_single(
                relabel_idx, relabel_idx_unlabeled, X_labeled, X_unlabeled, pred_probs, 
                pred_probs_unlabeled, extra_labels_single
            )

            np.save(f"results/{noise_rate}/single_{iter}.npy", single_model_accuracy_arr)